In [1]:
import pandas as pd
import pickle
import collections
import itertools


In [5]:
entrez_name = pd.read_csv('/scratch/PI/rbaltman/yash_margaret/gnbr_gene_nodes_vfinal.csv')
entrez_name['uri'] = entrez_name['uri'].apply(lambda x: x.split(':')[1])
entrez_name_dictionary = pd.Series(entrez_name.uri.values,index=entrez_name.name).to_dict()
entrez_name_dictionary

FileNotFoundError: File b'/scratch/PI/rbaltman/yash_margaret/gnbr_gene_nodes_vfinal.csv' does not exist

In [6]:
gene_mapping = pd.read_csv('gene_mapping.txt', sep = '\t')
gene_mapping = gene_mapping[pd.notnull(gene_mapping['NCBI Gene ID'])]
gene_mapping['NCBI Gene ID'] = gene_mapping['NCBI Gene ID'].astype(int)
gene_mapping_dict = pd.Series(gene_mapping['NCBI Gene ID'].values,index=gene_mapping['Approved symbol']).to_dict()

In [7]:
with open('gene_mapping_dict.pickle', 'wb') as handle:
    pickle.dump(gene_mapping_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# GWAS Data

In [9]:
alcohol = pd.read_csv('disease_GWAS_genes/alcohol_dependence_GWAS.csv')
alcohol

,Variant and risk allele,P-value,P-value annotation,RAF,OR,Beta,CI,Mapped gene,Reported trait,Trait(s),Study accession,Location
0,rs8062326-T,4 x 10-6,(BPALC),0.02,'-,'-,'-,AC099518.3,Alcohol dependence,alcohol dependence,GCST001211,16:19144694
1,rs1318937-?,3 x 10-6,(EA + AA),NR,'-,'-,'-,"SH3BP5-AS1, CAPN7",Alcohol dependence,alcohol dependence,GCST001254,3:15253857
2,rs10893366-?,8 x 10-6,(EA + AA),NR,'-,'-,'-,PKNOX2,Alcohol dependence,alcohol dependence,GCST001254,11:125308507
3,rs9636231-?,8 x 10-6,(EA + AA),NR,'-,'-,'-,"AHCYP4, AC062021.1",Alcohol dependence,alcohol dependence,GCST001254,2:138914385
4,rs237899-?,2 x 10-6,NaN,NR,1.69,'-,[1.36-2.10],"OXTR, CAV3",Depression and alcohol dependence,alcohol dependence,GCST001313,3:8766829
5,rs5968205-?,3 x 10-6,NaN,NR,2.2,'-,[1.58-3.07],"TERF1P4, AL121882.1",Depression and alcohol dependence,alcohol dependence,GCST001313,X:83699482
6,rs3827730-?,4 x 10-6,NaN,NR,1.72,'-,[1.36-2.16],FAF1,Depression and alcohol dependence,alcohol dependence,GCST001313,1:50472176
7,rs8111589-?,4 x 10-6,NaN,NR,1.64,'-,[1.33-2.03],OPA3,Depression and alcohol dependence,alcohol dependence,GCST001313,19:45531300
8,rs5922838-?,5 x 10-6,NaN,NR,2.17,'-,[1.55-3.02],"AL121882.1, TERF1P4",Depression and alcohol dependence,alcohol dependence,GCST001313,X:83616187
9,rs9805786-?,7 x 10-6,NaN,NR,1.65,'-,[1.33-2.05],"AL359736.1, SPATA13",Depression and alcohol dependence,alcohol dependence,GCST001313,13:24084217


In [8]:
def getGWASGeneSet(file):
    disease = pd.read_csv(file)
    disease_list = []
    for index, row in disease.iterrows():
        gene_list = []
        if "," in str(row["Mapped gene"]):
            gene_list = gene_list + (str(row["Mapped gene"]).split(","))
        else: 
            gene_list.append(row["Mapped gene"])
        disease_list.append(gene_list)
        
    disease_list = ([j for i in disease_list for j in i])
    for index, element in enumerate(disease_list):
        if "." in element:
            new = element.split(".")[0]
            disease_list[index] = new
    disease_list[:] = [s.replace(" ", "") for s in disease_list]
    
    disease_list = list(map(gene_mapping_dict.get, disease_list))
    disease_list = [x for x in disease_list if x is not None]
    
    return set(disease_list)

alcohol_GWAS = getGWASGeneSet('disease_GWAS_genes/alcohol_dependence_GWAS.csv')
bpd_GWAS = getGWASGeneSet('disease_GWAS_genes/bpd_GWAS.csv')
cannabis_GWAS = getGWASGeneSet('disease_GWAS_genes/cannabis_dependence_GWAS.csv')
cocaine_GWAS = getGWASGeneSet('disease_GWAS_genes/coacaine_dependence_GWAS.csv')
mdd_GWAS = getGWASGeneSet('disease_GWAS_genes/mdd_gwas.csv')
psychosis_GWAS = getGWASGeneSet('disease_GWAS_genes/psychosis_GWAS.csv')
scz_GWAS = getGWASGeneSet('disease_GWAS_genes/schizophrenia_GWAS.csv')

In [9]:
GWAS_dictionary = {}
GWAS_dictionary['Alcohol_use_disorders'] = alcohol_GWAS
GWAS_dictionary['Bipolar_disorders_and_related_disorders'] = bpd_GWAS
GWAS_dictionary['Cannabis_use_disorders'] = cannabis_GWAS
GWAS_dictionary['Cocaine_use_disorders'] = cocaine_GWAS
GWAS_dictionary['Depressive_disorders'] = mdd_GWAS
GWAS_dictionary['Drug-induced_psychosis'] = psychosis_GWAS
GWAS_dictionary['Schizophrenia_spectrum_and_other_psychotic_disorders'] = scz_GWAS
GWAS_dictionary['Substance_drug_induced_depressive_disorder'] = set()
GWAS_dictionary

{'Alcohol_use_disorders': {125,
  126,
  127,
  131,
  217,
  302,
  332,
  585,
  831,
  859,
  864,
  1233,
  1291,
  1600,
  1630,
  1879,
  2099,
  2104,
  2729,
  2774,
  2830,
  2869,
  3092,
  3295,
  3320,
  3329,
  3763,
  4045,
  4528,
  4589,
  4685,
  4703,
  4978,
  5021,
  5195,
  5521,
  5925,
  6005,
  6188,
  6445,
  6476,
  6536,
  6868,
  6885,
  8499,
  8829,
  8936,
  9215,
  9378,
  9674,
  9750,
  9759,
  10015,
  10150,
  10161,
  10317,
  10363,
  10371,
  10539,
  10734,
  10912,
  11124,
  11278,
  22990,
  22997,
  23261,
  23294,
  23328,
  23335,
  23473,
  23779,
  25890,
  25932,
  26010,
  26051,
  26227,
  26355,
  27253,
  27340,
  29035,
  29953,
  51752,
  53942,
  54072,
  54456,
  54622,
  54897,
  54933,
  54986,
  55117,
  55183,
  55287,
  55628,
  55686,
  55825,
  55848,
  56884,
  57007,
  57486,
  60370,
  63876,
  63923,
  64077,
  64785,
  79783,
  79785,
  80034,
  80207,
  83742,
  83988,
  84281,
  85004,
  90627,
  91526,
  92949,
  9

In [19]:
len(GWAS_dictionary['Schizophrenia_spectrum_and_other_psychotic_disorders'])

1261

# Psygenet

In [4]:
psygenet = pd.read_csv('diseaseSignatures/psygenet_v02.txt', sep = '\t')
psygenet

,Gene_Id,Gene_Symbol,Gene_Description,Disease_Id,DiseaseName,PsychiatricDisorder,Number_of_AbstractsValidated,Number_of_Abstracts,Source_Id,Score,Disease_Number
0,4852,NPY,neuropeptide Y,umls:C0001973,"alcoholic intoxication, chronic",Alcohol use disorders,5,23,psycur15,1.0,11
1,6532,SLC6A4,solute carrier family 6 (neurotransmitter tran...,umls:C0001973,"alcoholic intoxication, chronic",Alcohol use disorders,5,57,psycur15,1.0,23
2,6532,SLC6A4,solute carrier family 6 (neurotransmitter tran...,umls:C0005586,bipolar disorder,Bipolar disorders and related disorders,5,70,psycur15,1.0,23
3,6532,SLC6A4,solute carrier family 6 (neurotransmitter tran...,umls:C0011581,depressive disorder,Depressive disorders,5,337,psycur15,1.0,23
4,6532,SLC6A4,solute carrier family 6 (neurotransmitter tran...,umls:C0525045,mood disorders,Depressive disorders,5,86,psycur15,1.0,23
5,6532,SLC6A4,solute carrier family 6 (neurotransmitter tran...,umls:C1269683,major depressive disorder,Depressive disorders,5,134,psycur15,1.0,23
6,4988,OPRM1,"opioid receptor, mu 1",umls:C0001973,"alcoholic intoxication, chronic",Alcohol use disorders,5,57,psycur15,1.0,8
7,2555,GABRA2,"gamma-aminobutyric acid (GABA) A receptor, alp...",umls:C0001973,"alcoholic intoxication, chronic",Alcohol use disorders,5,51,psycur15,1.0,2
8,1392,CRH,corticotropin releasing hormone,umls:C0011581,depressive disorder,Depressive disorders,5,68,psycur15,1.0,10
9,1312,COMT,catechol-O-methyltransferase,umls:C0005586,bipolar disorder,Bipolar disorders and related disorders,5,55,psycur15,1.0,12


In [30]:
psygenet.PsychiatricDisorder = psygenet.PsychiatricDisorder.str.replace(' ', '_')
psygenet.PsychiatricDisorder = psygenet.PsychiatricDisorder.str.replace('/', '_')

psygenet

,Gene_Id,Gene_Symbol,Gene_Description,Disease_Id,DiseaseName,PsychiatricDisorder,Number_of_AbstractsValidated,Number_of_Abstracts,Source_Id,Score,Disease_Number
0,4852,NPY,neuropeptide Y,umls:C0001973,"alcoholic intoxication, chronic",Alcohol_use_disorders,5,23,psycur15,1.0,11
1,6532,SLC6A4,solute carrier family 6 (neurotransmitter tran...,umls:C0001973,"alcoholic intoxication, chronic",Alcohol_use_disorders,5,57,psycur15,1.0,23
2,6532,SLC6A4,solute carrier family 6 (neurotransmitter tran...,umls:C0005586,bipolar disorder,Bipolar_disorders_and_related_disorders,5,70,psycur15,1.0,23
3,6532,SLC6A4,solute carrier family 6 (neurotransmitter tran...,umls:C0011581,depressive disorder,Depressive_disorders,5,337,psycur15,1.0,23
4,6532,SLC6A4,solute carrier family 6 (neurotransmitter tran...,umls:C0525045,mood disorders,Depressive_disorders,5,86,psycur15,1.0,23
5,6532,SLC6A4,solute carrier family 6 (neurotransmitter tran...,umls:C1269683,major depressive disorder,Depressive_disorders,5,134,psycur15,1.0,23
6,4988,OPRM1,"opioid receptor, mu 1",umls:C0001973,"alcoholic intoxication, chronic",Alcohol_use_disorders,5,57,psycur15,1.0,8
7,2555,GABRA2,"gamma-aminobutyric acid (GABA) A receptor, alp...",umls:C0001973,"alcoholic intoxication, chronic",Alcohol_use_disorders,5,51,psycur15,1.0,2
8,1392,CRH,corticotropin releasing hormone,umls:C0011581,depressive disorder,Depressive_disorders,5,68,psycur15,1.0,10
9,1312,COMT,catechol-O-methyltransferase,umls:C0005586,bipolar disorder,Bipolar_disorders_and_related_disorders,5,55,psycur15,1.0,12


In [37]:
psygenet_dictionary = {k: set(g["Gene_Id"].tolist()) for k,g in psygenet.groupby("PsychiatricDisorder")}
# with open('psygenet_dictionary.pickle', 'wb') as handle:
#     pickle.dump(psygenet_dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Disease Signature

In [42]:
diseaseSig = {}
for k in psygenet_dictionary.keys():
    diseaseSig[k] = psygenet_dictionary[k].union(GWAS_dictionary[k])
    
# with open('diseaseSig_psygenet_GWAS.pickle', 'wb') as handle:
#     pickle.dump(diseaseSig, handle, protocol=pickle.HIGHEST_PROTOCOL)